In [ ]:
import numpy as np
from my_functions import *
import matplotlib.pyplot as plt
import pickle as pkl

In [ ]:
## Load the catalog
with open('pkl/cat_flambda_photoz_gaia.pkl', 'rb') as file:
    cat = pkl.load(file)

In [ ]:
pm_flx = cat['flx'].T * 1e-19
pm_err = cat['flx_err'].T

In [ ]:
err_fit_params = []
for nb_c in range(60):
    mask = (pm_flx[nb_c] > 0) & (pm_err[nb_c] > 0)
    err_fit_params.append(
        np.polyfit(np.log10(pm_flx[nb_c, mask]), np.log10(pm_err[nb_c, mask]), 1)
    )

In [ ]:
fig, ax = plt.subplots(figsize=(11, 9))

nb_c = 12
m = err_fit_params[nb_c][0]
b = err_fit_params[nb_c][1]

ax.scatter(np.log10(np.abs(pm_flx[nb_c])), np.log10(np.abs(pm_err[nb_c])))
x_Arr = np.linspace(-22, -14, 100)
ax.plot(x_Arr, b + m*x_Arr, c='C1')

plt.show()

In [ ]:
np.save('npy/err_fit_params_minijpas', np.array(err_fit_params))